In [1]:
import sqlalchemy
import pandas as pd
from pivottablejs import pivot_ui

from bokeh.plotting import figure, output_notebook, show
import bokeh.charts as charts
import bokeh.layouts as layouts
import bokeh.models.widgets as widgets


# import plotly
# # import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# plotly.offline.init_notebook_mode(connected=True)

# Para ter tabela dinamica
from pivottablejs import pivot_ui


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
conn = 'mysql+pymysql://root@127.0.0.1:3306/tblai'

In [4]:
querizona_pedidos = r"""
SELECT pedidos.*,
       usuarios.Codigo,
       usuarios.Nome AS usuario_nome,
       agentes.*,
       agentes.Nome AS agente_nome,
       status_pedido.Nome AS status_final_pedido_nome
FROM pedidos,
     usuarios,

  (SELECT agentes.Nome,
          agentes.Codigo,
          uf.Nome AS agente_UF,
          tipo_poder.Nome AS agente_Poder,
          tipo_nivel_federativo.Nome AS agente_Nivel_Federativo
   FROM agentes,
        uf,
        tipo_poder,
        tipo_nivel_federativo
   WHERE agentes.CodigoUF = uf.Codigo
     AND agentes.CodigoPoder = tipo_poder.Codigo
     AND agentes.CodigoNivelFederativo = tipo_nivel_federativo.Codigo ) AS agentes,
     status_pedido
WHERE pedidos.CodigoUsuario = usuarios.Codigo
  AND pedidos.CodigoAgente = agentes.Codigo
  AND pedidos.CodigoStatusPedido = status_pedido.Codigo;

"""

pedidos = pd.read_sql_query(querizona_pedidos, conn)
# pedidos.to_feather('pedidos.feather')

In [6]:
pedidos_por_data = pedidos.groupby(pedidos.DataEnvio).size()

In [7]:
pedidos.groupby( [pedidos.agente_Poder,pedidos.agente_UF]).size()

agente_Poder  agente_UF          
Executivo     Espírito Santo             1
              Maranhão                   2
              Mato Grosso                1
              Minas Gerais               1
              Paraná                    78
              Rio de Janeiro             2
              São Paulo              14944
Judiciário    Acre                       5
              Alagoas                    5
              Amapá                     10
              Amazonas                   5
              Bahia                      5
              Ceará                      5
              Distrito Federal           5
              Espírito Santo             5
              Goiás                      5
              Maranhão                  11
              Mato Grosso                5
              Mato Grosso do Sul         5
              Minas Gerais               5
              Paraíba                    5
              Pernambuco                 5
              Piauí 

In [16]:
figura_pedidos_por_data = charts.TimeSeries(pedidos_por_data, xlabel='Data', 
                                            ylabel='Pedidos',
                                            width=900,
                                            title='Quantidade de Pedidos por Data')
show(figura_pedidos_por_data)

In [68]:
# Vamos filtrar por tipo de Poder, e dentro disso, por UF

pedidos_por_poder = {'Executivo': pedidos[pedidos.agente_Poder == 'Executivo'].groupby('DataEnvio').size(),
                     'Judiciário': pedidos[pedidos.agente_Poder == 'Judiciário'].groupby('DataEnvio').size(),
                     'Legislativo': pedidos[pedidos.agente_Poder == 'Legislativo'].groupby('DataEnvio').size(),
                     'Total': pedidos.groupby('DataEnvio').size()}

show(charts.TimeSeries(pedidos_por_poder))


In [62]:
pedidos.agente_Poder.unique()

array(['Judiciário', 'Executivo', 'Legislativo'], dtype=object)

In [66]:
pedidos[pedidos.agente_Poder == 'Executivo'].groupby('DataEnvio').size()

DataEnvio
2012-05-16     1
2012-08-24     1
2012-08-27     1
2012-08-31     1
2012-09-01     1
2012-09-03     3
2012-09-04     1
2012-09-06     1
2012-09-09     1
2012-09-10     1
2012-09-11     3
2012-09-13     3
2012-09-17     4
2012-09-18     9
2012-09-19    10
2012-09-20    14
2012-09-21     8
2012-09-23     1
2012-09-24     2
2012-09-26     1
2012-09-27    10
2012-09-28     9
2012-09-30     2
2012-10-01     1
2012-10-02     1
2012-10-03     1
2012-10-04    13
2012-10-05     2
2012-10-06     2
2012-10-08     4
              ..
2016-09-21     8
2016-09-22     9
2016-09-23     3
2016-09-25     1
2016-09-26     5
2016-09-27     2
2016-09-28     2
2016-09-29     2
2016-09-30     1
2016-10-25     1
2016-11-09     1
2016-11-17     1
2016-11-25     1
2016-12-13     1
2017-01-13     1
2017-02-06     1
2017-02-10     1
2017-02-15     2
2017-02-16     1
2017-03-06     1
2017-03-08     1
2017-03-10     4
2017-03-17     1
2017-03-21     1
2017-03-30     2
2017-04-06     1
2017-04-13     1
2017

In [5]:
# Planilha de pedidos para mandar para o DataFrame
colunas_de_codigos = [x for x in pedidos.columns if 'Codigo' in x]

pedidos_para_exportar = pedidos.drop(['Titulo','Descricao','Slug','Link', 'CriadoExternamente', 'IdentificadorExterno', 'Criacao', 'Alteracao', *colunas_de_codigos], axis=1)

# pedidos_para_exportar['ano'] = pedidos_para_exportar['DataEnvio'].apply(lambda x: x.year)


In [6]:
pedidos_para_exportar['ano'] = pedidos_para_exportar['DataEnvio'].apply(lambda x: str(x.year))

/Users/tiago/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
pedidos_para_exportar.to_json('pedidos.json', orient='records')

In [11]:
pivot_ui(pedidos_para_exportar)

In [15]:
pedidos_para_exportar.to_json('pedidos.json', orient='records')

ValueError: DataFrame columns must be unique for orient='records'.